# Test and Trial on Gaussian Mixture Distribution.

## Functions

In [ ]:
import sys
import numpy as np
import tensorflow as tf
from tensorflow.contrib.distributions import (
    Categorical, NormalWithSoftplusScale, Mixture)

from nn4post import InferenceBuilder
try:
    from tensorflow.contrib.distributions import Independent
except:
    print('WARNING - Your TF < 1.4.0.')
    from nn4post.utils.independent import Independent

In [ ]:
# For testing (and debugging)
SEED = 123456
tf.set_random_seed(SEED)
np.random.seed(SEED)

In [ ]:
def make_log_posterior(target_c, target_mu, target_zeta):
  
    target_c = tf.convert_to_tensor(target_c)
    target_mu = tf.convert_to_tensor(target_mu)
    target_zeta = tf.convert_to_tensor(target_zeta)

    # -- Gaussian Mixture Distribution
    with tf.name_scope('posterior'):

      cat = Categorical(probs=target_c)
      components = [
          Independent(
              NormalWithSoftplusScale(target_mu[i], target_zeta[i])
          ) for i in range(target_c.shape[0])
      ]
      p = Mixture(cat, components)

      def log_posterior(theta):
          return p.log_prob(theta)

    return log_posterior

In [ ]:
def shall_stop(loss_values, tolerance, n_means=20):

    if len(loss_values) < 2 * n_means:

        return False

    else:
        previous_loss = np.mean(loss_values[-2*n_means:-n_means])
        current_loss = np.mean(loss_values[-n_means:])
        delta_loss = previous_loss - current_loss
        relative_delta_loss = abs( delta_loss / (current_loss + 1e-8) )

        if relative_delta_loss < tolerance:
            return True
        else:
            return False


def test(target_c, target_mu, target_zeta,
         init_var, tolerance=1e-2):

    tf.reset_default_graph()

    log_p = make_log_posterior(target_c, target_mu, target_zeta)

    n_c, n_d = init_var['mu'].shape
    ib = InferenceBuilder(n_c, n_d, log_p)
    
    a = tf.Variable(init_var['a'], dtype='float32')
    mu = tf.Variable(init_var['mu'], dtype='float32')
    zeta = tf.Variable(init_var['zeta'], dtype='float32')
    loss, gradients = ib.make_loss_and_gradients(a, mu, zeta)

    optimizer = tf.train.AdamOptimizer(0.01)
    train_op = optimizer.apply_gradients(gradients)
    
    init = tf.global_variables_initializer()

    with tf.Session() as sess:
        
        sess.run(init)

        test_result = {'loss': [], 'a': [], 'mu': [], 'zeta': []}
        step = 0

        while not shall_stop(test_result['loss'], tolerance):

            _, loss_val, a_val, mu_val, zeta_val \
                = sess.run([train_op, loss, a, mu, zeta])

            test_result['loss'].append(loss_val)
            test_result['a'].append(a_val)
            test_result['mu'].append(mu_val)
            test_result['zeta'].append(zeta_val)
            
            step += 1
            if (step+1) % 100 == 0:
                print(step, loss_val)

    return test_result

In [ ]:
# Helpers

def softplus(x, limit=10.):
    return np.where(x<limit, np.log(1. + np.exp(x)), x)


def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

## Trials

In [ ]:
n_d = 2
ones = np.ones([n_d]).astype('float32')
target_c = np.array([0.7, 0.25, 0.05]).astype('float32')
target_mu = np.array([-3*ones, 0*ones, 3*ones]).astype('float32')
target_zeta = np.array([ones, ones, ones]).astype('float32')

n_c = 5
init_var = {
    'a': np.zeros([n_c]).astype('float32'),
    'mu': np.random.normal(0., 5., size=[n_c, n_d]).astype('float32'),
    'zeta': np.zeros([n_c, n_d]).astype('float32'),
}

test_result = test(target_c, target_mu, target_zeta,
                   init_var, tolerance=1e-3)

In [ ]:
print(test_result['loss'])

In [ ]:
print(softmax(test_result['a'][-1]), '\n')
print(test_result['mu'][-1], '\n')
print(test_result['zeta'][-1], '\n')